In [ ]:

# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Agent - Orchestration with LLM

## Set configuration

In [2]:
# @title Package install
%pip install --upgrade --quiet --user google-cloud-aiplatform

In [3]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

In [7]:
# @title Define constants

PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

In [4]:
# @title Import packages

import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding

In [8]:
# @title Initialize Vertex AI

# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

In [11]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을 참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 제품 추천 : http://sec_vd.com/제품검색
  2. 대리점 추천 : http://sec_vd.com/대리점추천
  3. 번역테스트 : http://sec_vd.com/번역테스트
  4. 대리점 수리접수 : http://sec_vd.com/대리점수리접수
  5. 기타 : http://sec_vd.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def get_prompts(chat_history, query):

  prompt_1 =f"""
          당신은 삼성 제품 추천을 위한 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 삼성 제품 추천에 대해서 제품명 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 추천을 하는 지역위치는 최신 대화의 지역을 우선으로 추천해주세요.
          3. 질문 또는 대화내용에서 특정 위치를 알려주지 않으면 기본값으로 서울 지역을 기반으로 답해주세요.
          4. 사전 훈련된 지식에서 질문하는 지역에서 자주 언급되는 제품을 이용해서 추천해주시고, 이유를 알려주세요.
          5. 자세한 답변을 위해서 추가적으로 사용자의 관심사항(색상,크기,기능등을) 요청해주세요.
          6. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_2 = f"""당신은 삼성 가전 제품 사용자를 위해 번역해주는 AI 어시스턴트 입니다. 아래 내용에 맞게 번역해주세요.

          1. 번역 대상 언어 선택은 가장 최근 대화를 참고해서 선택하고, 없다면 번역대상 언어 지정을 사용자에게 요청해주세요.
          2. 번역을 해야할 부분은 요청하는 문맥에 맞게 선택해서 번역해주세요.
          3. 특별하게 번역해야 할 부분을 제시하지 않으면 전체 문장을 번역해주세요.
          4. 답변은 최대한 간략하게 번역된 부분만 표현하고 설명하지 말아주세요.

          <번역을 위한 질문> : {query}

  """

  prompt_3 = f"""당신은 삼성 제품 사용자를 위해 삼성 대리점 위치 추천해주는 AI Assistant입니다.

          1. 질문에 맞는 삼성제품을 구매할 수 있는 대리점 명을 3~4개 정도 추천해주세요.
          2. 해당 대리점의 주소를 알려주고 왜 추천하는지 알려주세요.
          3. 전체적인 답변은 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_4 = f"""당신은 삼성제품 사용자를 위해 대리점 수리 접수를 도와주는 AI Assistant입니다.

          1. 수리접수를 위한 수리점 위치는 최신 대화의 지역을 우선으로 구글에서 검색해서 추천해주세요.
          2. 질문 또는 대화내용에서 특정 위치를 알려주지 않으면 기본값으로 서울 지역을 기반으로 답해주세요.
          3. 수리점의 정확한 주소와 전화번호를 같이 보여주세요.
          4. 만일 정확한 지역정보 또는 수리점 위치를 모를 때는 해당 위치정보를 요청해주세요.
          5. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_5 = f"""당신은 삼성제품 사용자의 질문에 답변을 해주는 AI Assistant입니다.
          사용자의 다양한 질문에 대해서 검색 결과를 활용해서 답변 해주세요.
          가급적 답변은 10줄 이내로 짧고 간략하게 답해주세요.
          <질문> : {query}

  """

  classified = query_classification(chat_history, query)

  task = eval(classified)['classification']['class']
  print(f"Selected Task : {task}")
  if task == '제품 추천':
    return prompt_1, True
  elif task == '번역테스트':
    return prompt_2, False
  elif task == '대리점 추천':
    return prompt_3, True
  elif task == '대리점 수리접수':
    return prompt_4, True
  elif task == '기타':
    return prompt_5, True
  else:
    return prompt_5, True


In [12]:
# @title AI Agent

chat = model.start_chat()

search = True

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  prompt, search = get_prompts(chat_history, query)

  response = interactive_chat(chat, prompt, search)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 안녕하세요.
Selected Task : 기타


AI Agent : 안녕하세요! 삼성 제품 사용에 대한 질문이 있으시면 언제든지 편하게 물어보세요.  최선을 다해 도와드리겠습니다.


------------------------------------ 
사용자: 삼성 냉장고는 어디서 사야 하나요 ?
Selected Task : 대리점 추천


AI Agent : 삼성 냉장고는 삼성 디지털프라자, 백화점, 또는 온라인 삼성닷컴에서 구매하실 수 있습니다.  정확한 대리점 위치는 거주 지역에 따라 다르므로, 삼성전자 홈페이지의 매장 찾기 기능을 이용하시면 가까운 곳을 찾으실 수 있습니다.  

추천 대리점은 다음과 같습니다. (단,  구체적인 주소와 추천 이유는 제공된 정보만으로는 확인이 불가능합니다.  가까운 매장을 삼성전자 홈페이지에서 확인해주세요.)

1. 삼성 디지털프라자 (가장 가까운 지점) : 다양한 삼성 제품과 전문 상담을 받을 수 있습니다.
2. 롯데백화점 (가전 매장이 있는 지점) :  다른 브랜드 제품과 비교하며 쇼핑 가능합니다.
3.  온라인 삼성닷컴: 편리하게 온라인으로 구매 가능하며 다양한 할인 혜택을 확인할 수 있습니다.

보다 정확한 정보를 위해서는  삼성전자 홈페이지의 매장찾기 서비스를 이용하는 것이 좋습니다.


------------------------------------ 
사용자: 종료
